In [1]:
import re
def token(sentence, remove_vowels=False, remove_repeat=False, minchars=2):
    tokens = []
#   for t in re.findall("[A-Z]{2,}(?![a-z])|[A-Z][a-z]+(?=[A-Z])|[\w]+",sentence.lower()):
    for t in re.findall("[a-zA-Z]+",sentence.lower()):

        if len(t)>=minchars:
            if remove_vowels:
                t=removeVovels(t)
            if remove_repeat:
                t=removeRepeat(t)
            tokens.append(t)
    return tokens

VOWELS = ['a', 'e', 'i', 'o', 'u']

def removeRepeat(string):
    return re.sub(r'(.)\1+', r'\1\1', string)     

def removeVovels(string):
    return ''.join([l for l in string.lower() if l not in VOWELS])

if __name__ == '__main__':
    pass

def normalize_matrix(matrix):
    pass

In [2]:
import numpy as np
import h5py
import pickle
from copy import deepcopy
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing import sequence
from keras import backend as K
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.utils import np_utils

Using TensorFlow backend.


In [3]:
################# GLOBAL VARIABLES #####################
#Filenames
#TODO: Add to coding conventions that directories are to always end with '/'
Masterdir = 'C:/Users/DEEPAK KUMAR/Sub-word-LSTM-master'
Datadir = 'Data/'
Modeldir = 'Models/'
Featuredir = 'Features/'
inputdatasetfilename = 'IIITH_Codemixed.txt'
exp_details = 'new_experiment'

#Data I/O formatting
SEPERATOR = '\t'
DATA_COLUMN = 1
LABEL_COLUMN = 3
LABELS = ['0','1','2'] # 0 -> Negative, 1-> Neutral, 2-> Positive
mapping_char2num = {}
mapping_num2char = {}
MAXLEN = 200

#LSTM Model Parameters
#Embedding
MAX_FEATURES = 0
embedding_size = 128
# Convolution
filter_length = 3
nb_filter = 128
pool_length = 3
# LSTM
lstm_output_size = 128
# Training
batch_size = 128
number_of_epochs = 50
numclasses = 3
test_size = 0.2
########################################################

In [4]:
def parse(Masterdir,filename,seperator,datacol,labelcol,labels):
	"""
	Purpose -> Data I/O
	Input   -> Data file containing sentences and labels along with the global variables
	Output  -> Sentences cleaned up in list of lists format along with the labels as a numpy array
	"""
	#Reads the files and splits data into individual lines
	f=open("IIITH_Codemixed.txt",'r',encoding='utf-8')
	lines = f.read().lower()
	lines = lines.lower().split('\n')[:-1]

	X_train = []
	Y_train = []
	
	#Processes individual lines
	for line in lines:
		# Seperator for the current dataset. Currently '\t'. 
		line = line.split(seperator)
		#Token is the function which implements basic preprocessing as mentioned in our paper
		tokenized_lines = token(line[datacol])
		
		#Creates character lists
		char_list = []
		for words in tokenized_lines:
			for char in words:
				char_list.append(char)
			char_list.append(' ')
		#print(char_list) - Debugs the character list created
		X_train.append(char_list)
		
		#Appends labels
		if line[labelcol] == labels[0]:
			Y_train.append(0)
		if line[labelcol] == labels[1]:
			Y_train.append(1)
		if line[labelcol] == labels[2]:
			Y_train.append(2)
	
	#Converts Y_train to a numpy array	
	Y_train = np.asarray(Y_train)
	assert(len(X_train) == Y_train.shape[0])

	return [X_train,Y_train]

In [5]:
def convert_char2num(mapping_n2c,mapping_c2n,trainwords,maxlen):
	"""
	Purpose -> Convert characters to integers, a unique value for every character
	Input   -> Training data (In list of lists format) along with global variables
	Output  -> Converted training data along with global variables
	"""
	allchars = []
	errors = 0

	#Creates a list of all characters present in the dataset
	for line in trainwords:
		try:
			allchars = set(allchars+line)
			allchars = list(allchars)
		except:
			errors += 1

	#print(errors) #Debugging
	#print(allchars) #Debugging 

	#Creates character dictionaries for the characters
	charno = 0
	for char in allchars:
		mapping_char2num[char] = charno
		mapping_num2char[charno] = char
		charno += 1

	assert(len(allchars)==charno) #Checks

	#Converts the data from characters to numbers using dictionaries 
	X_train = []
	for line in trainwords:
		char_list=[]
		for letter in line:
			char_list.append(mapping_char2num[letter])
		#print(no) -- Debugs the number mappings
		X_train.append(char_list)
	print(mapping_char2num)
	print(mapping_num2char)
	#Pads the X_train to get a uniform vector
	#TODO: Automate the selection instead of manual input
	X_train = sequence.pad_sequences(X_train[:], maxlen=maxlen)
	return [X_train,mapping_num2char,mapping_char2num,charno]

In [6]:
def RNN(X_train,y_train,args):
	"""
	Purpose -> Define and train the proposed LSTM network
	Input   -> Data, Labels and model hyperparameters
	Output  -> Trained LSTM network
	"""
	#Sets the model hyperparameters
	#Embedding hyperparameters
	max_features = args[0]
	maxlen = args[1]
	embedding_size = args[2]
	# Convolution hyperparameters
	filter_length = args[3]
	nb_filter = args[4]
	pool_length = args[5]
	# LSTM hyperparameters
	lstm_output_size = args[6]
	# Training hyperparameters
	batch_size = args[7]
	nb_epoch = args[8]
	numclasses = args[9]
	test_size = args[10] 

	#Format conversion for y_train for compatibility with Keras
	y_train = np_utils.to_categorical(y_train, numclasses) 
	#Train & Validation data splitting
	X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=test_size, random_state=42)
	
	#Build the sequential model
	# Model Architecture is:
	# Input -> Embedding -> Conv1D+Maxpool1D -> LSTM -> LSTM -> FC-1 -> Softmaxloss
	print('Build model...')
	model = Sequential()
	model.add(Embedding(max_features, embedding_size, input_length=maxlen))
	model.add(Convolution1D(filters=128,kernel_size=3,
							
							padding='valid',
							activation='relu',
							dilation_rate=1))
	model.add(MaxPooling1D(pool_length=pool_length))
	model.add(LSTM(lstm_output_size, dropout=0.2, return_sequences=True))
	model.add(LSTM(lstm_output_size, dropout=0.2, return_sequences=False))
	model.add(Dense(numclasses))
	model.add(Activation('softmax'))

	# Optimizer is Adamax along with categorical crossentropy loss
	model.compile(loss='categorical_crossentropy',
			  	optimizer='adamax',
			  	metrics=['accuracy'])
	

	print('Train...')
	#Trains model for 50 epochs with shuffling after every epoch for training data and validates on validation data
	model.fit(X_train, y_train, 
			  batch_size=batch_size, 
			  shuffle=True, 
			  nb_epoch=nb_epoch,
			  validation_data=(X_valid, y_valid))
	return model

In [7]:
# def save_model(Masterdir,filename,model):
# 	"""
# 	Purpose -> Saves Keras model files to the given directory
# 	Input   -> Directory and experiment details to be saved and trained model file
# 	Output  -> Nil
# 	"""
# 	#Referred from:- http://keras.io/getting-started/faq/#how-can-i-save-a-keras-model
# 	model.save_weights(Masterdir+'Models/LSTM_'+filename+'_weights.h5')
# 	json_string = model.to_json()
# 	f = open(Masterdir+'Models/'+'LSTM_'+filename+'_architecture.json','w')
# 	f.write(json_string)
# 	f.close()

In [8]:
# def get_activations(model, layer, X_batch):
# 	"""
# 	Purpose -> Obtains outputs from any layer in Keras
# 	Input   -> Trained model, layer from which output needs to be extracted & files to be given as input
# 	Output  -> Features from that layer 
# 	"""
# 	#Referred from:- TODO: Enter the forum link from where I got this
# 	get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output,])
# 	activations = get_activations([X_batch,0])
# 	return activations

In [9]:
def evaluate_model(X_test,y_test,model,batch_size,numclasses):
	"""
	Purpose -> Evaluate any model on the testing data
	Input   -> Testing data and labels, trained model and global variables
	Output  -> Nil
	"""
	#Convert y_test to one-hot encoding
	y_test = np_utils.to_categorical(y_test, numclasses)
	#Evaluate the accuracies
	score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)
	print('Test score:', score)
	print('Test accuracy:', acc)

In [10]:
# def save_data(Masterdir,filename,X_train,X_test,y_train,y_test,features_train,features_test):
# 	"""
# 	Purpose -> Saves train, test data along with labels and features in the respective directories in the folder
# 	Input   -> Train and test data, labels and features along with the directory and experiment details to be mentioned
# 	Output  -> Nil
# 	"""
# 	h5f = h5py.File(Masterdir+Datadir+'Xtrain_'+filename+'.h5', 'w')
# 	h5f.create_dataset('dataset', data=X_train)
# 	h5f.close()

# 	h5f = h5py.File(Masterdir+Datadir+'Xtest_'+filename+'.h5', 'w')
# 	h5f.create_dataset('dataset', data=X_test)
# 	h5f.close()

# 	output = open(Masterdir+Datadir+'Ytrain_'+filename+'.pkl', 'wb')
# 	pickle.dump([y_train], output)
# 	output.close()

# 	output = open(Masterdir+Datadir+'Ytest_'+filename+'.pkl', 'wb')
# 	pickle.dump([y_test], output)
# 	output.close()

# 	h5f = h5py.File(Masterdir+Featuredir+'features_train_'+filename+'.h5', 'w')
# 	h5f.create_dataset('dataset', data=features_train)
# 	h5f.close()

# 	h5f = h5py.File(Masterdir+Featuredir+'features_test_'+filename+'.h5', 'w')
# 	h5f.create_dataset('dataset', data=features_test)
# 	h5f.close()

In [11]:
if __name__ == '__main__':
	"""
	Master function
	"""
	print('Starting RNN Engine...\nModel: Char-level LSTM.\nParsing data files...')
	out = parse(Masterdir,inputdatasetfilename,SEPERATOR,DATA_COLUMN,LABEL_COLUMN,LABELS)
	X_train = out[0]
	y_train = out[1]
	print('Parsing complete!')

	print('Creating character dictionaries and format conversion in progess...')
	out = convert_char2num(mapping_num2char,mapping_char2num,X_train,MAXLEN)
	mapping_num2char = out[1]
	mapping_char2num = out[2]
	MAX_FEATURES = out[3]
	X_train = np.asarray(out[0])
	y_train = np.asarray(y_train).flatten()
	print('Complete!')
	
	print('Splitting data into train and test...')
	X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
	print('X_train shape:', X_train.shape)
	print('X_test shape:', X_test.shape)
	
	print('Creating LSTM Network...')
	model = RNN(deepcopy(X_train),deepcopy(y_train),[MAX_FEATURES, MAXLEN, embedding_size,\
			     filter_length, nb_filter, pool_length, lstm_output_size, batch_size, \
			     number_of_epochs, numclasses, test_size])

	print('Evaluating model...')
	evaluate_model(X_test,deepcopy(y_test),model,batch_size,numclasses)
	
# 	print('Feature extraction pipeline running...')
# 	activations = get_activations(model, 4, X_train)
# 	features_train = np.asarray(activations)
# 	activations = get_activations(model, 4, X_test)
# 	features_test = np.asarray(activations)
# 	print('Features extracted!')
	
# 	print('Saving experiment...')
# 	save_model(Masterdir,exp_details,model)
# 	save_data(Masterdir,exp_details,X_train,X_test,y_train,y_test,features_train,features_test)
# 	print('Saved! Experiment finished!')

Starting RNN Engine...
Model: Char-level LSTM.
Parsing data files...
Parsing complete!
Creating character dictionaries and format conversion in progess...
{'x': 0, 'm': 1, 'a': 2, 'p': 3, 'f': 4, 'n': 5, 'd': 6, 'u': 7, 'y': 8, 'q': 9, 'r': 10, 'c': 11, 'v': 12, 'l': 13, 'j': 14, 'h': 15, ' ': 16, 'z': 17, 'k': 18, 'e': 19, 't': 20, 's': 21, 'w': 22, 'i': 23, 'g': 24, 'o': 25, 'b': 26}
{0: 'x', 1: 'm', 2: 'a', 3: 'p', 4: 'f', 5: 'n', 6: 'd', 7: 'u', 8: 'y', 9: 'q', 10: 'r', 11: 'c', 12: 'v', 13: 'l', 14: 'j', 15: 'h', 16: ' ', 17: 'z', 18: 'k', 19: 'e', 20: 't', 21: 's', 22: 'w', 23: 'i', 24: 'g', 25: 'o', 26: 'b'}
Complete!
Splitting data into train and test...
X_train shape: (3103, 200)
X_test shape: (776, 200)
Creating LSTM Network...
Build model...
Instructions for updating:
Colocations handled automatically by placer.


C:\Users\DEEPAK KUMAR\Anaconda3\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=3)`


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train...
Instructions for updating:
Use tf.cast instead.


C:\Users\DEEPAK KUMAR\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 2482 samples, validate on 621 samples
Epoch 1/50
2482/2482 [==============================] - 17s 7ms/step - loss: 0.9904 - acc: 0.5052 - val_loss: 0.9961 - val_acc: 0.4815
Epoch 2/50
2482/2482 [==============================] - 9s 4ms/step - loss: 0.9671 - acc: 0.5097 - val_loss: 0.9832 - val_acc: 0.4928
Epoch 3/50
2482/2482 [==============================] - 9s 4ms/step - loss: 0.9455 - acc: 0.5254 - val_loss: 0.9625 - val_acc: 0.4767
Epoch 4/50
2482/2482 [==============================] - 9s 4ms/step - loss: 0.9172 - acc: 0.5479 - val_loss: 0.9372 - val_acc: 0.5427
Epoch 5/50
2482/2482 [==============================] - 9s 3ms/step - loss: 0.8883 - acc: 0.5898 - val_loss: 0.9098 - val_acc: 0.5475
Epoch 6/50
2482/2482 [==============================] - 9s 3ms/step - loss: 0.8752 - acc: 0.5838 - val_loss: 0.8981 - val_acc: 0.5700
Epoch 7/50
2482/2482 [==============================] - 9s 3ms/step - loss: 0.8483 - acc: 0.6035 - val_loss: 0.8926 - val_acc: 0.5749
Epoch 8/50
248

In [13]:
# print('Saving experiment...')
# save_model(Masterdir,exp_details,model)
# save_data(Masterdir,exp_details,X_train,X_test,y_train,y_test,features_train,features_test)
# print('Saved! Experiment finished!')